In [ ]:
# DistilBERT Classification - Suite du preprocessing
# Ce notebook continue après le preprocessing des données

# ============================================================================
# SECTION 1 : Imports et configuration
# ============================================================================

# Installation des packages nécessaires (décommenter si besoin)
# !pip install transformers torch datasets scikit-learn

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    DistilBertTokenizer,
    DistilBertForSequenceClassification,
    AdamW,
    get_linear_schedule_with_warmup
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import numpy as np

# Configuration du device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Utilisation du device: {device}")

# ============================================================================
# SECTION 2 : Préparation des données depuis df (continuité du preprocessing)
# ============================================================================

# On récupère les colonnes 'text' et 'label_encoded' du df créé précédemment
# Assurez-vous d'avoir exécuté le notebook de preprocessing avant !

print(f"\nNombre total d'échantillons: {len(df)}")
print(f"\nDistribution des classes:")
print(df['label_encoded'].value_counts())

# Mapping des labels
label_names = {0: 'Politics', 1: 'Sport', 2: 'Tech'}

# ============================================================================
# SECTION 3 : Division Train/Validation/Test
# ============================================================================

# Division train/temp (80/20)
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    df['text'].values,
    df['label_encoded'].values,
    test_size=0.2,
    random_state=42,
    stratify=df['label_encoded'].values
)

# Division temp en val/test (50/50 de temp = 10/10 du total)
val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts,
    temp_labels,
    test_size=0.5,
    random_state=42,
    stratify=temp_labels
)

print(f"\n📊 Division des données:")
print(f"  Train: {len(train_texts)} échantillons ({len(train_texts)/len(df)*100:.1f}%)")
print(f"  Validation: {len(val_texts)} échantillons ({len(val_texts)/len(df)*100:.1f}%)")
print(f"  Test: {len(test_texts)} échantillons ({len(test_texts)/len(df)*100:.1f}%)")

# ============================================================================
# SECTION 4 : Classe Dataset PyTorch personnalisée
# ============================================================================

class NewsDataset(Dataset):
    """Dataset PyTorch pour les articles de presse"""

    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        # Tokenization avec DistilBERT
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

print("\n✅ Classe Dataset créée")

# ============================================================================
# SECTION 5 : Initialisation du tokenizer et création des datasets
# ============================================================================

print("\n🔄 Chargement du tokenizer DistilBERT...")
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Création des datasets
print("📦 Création des datasets...")
train_dataset = NewsDataset(train_texts, train_labels, tokenizer)
val_dataset = NewsDataset(val_texts, val_labels, tokenizer)
test_dataset = NewsDataset(test_texts, test_labels, tokenizer)

print("✅ Datasets créés avec succès!")

# ============================================================================
# SECTION 6 : Configuration des hyperparamètres
# ============================================================================

# Hyperparamètres
BATCH_SIZE = 16
EPOCHS = 3
LEARNING_RATE = 2e-5
NUM_LABELS = 3
MAX_LENGTH = 512

print(f"\n⚙️ Hyperparamètres:")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Epochs: {EPOCHS}")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  Max length: {MAX_LENGTH}")

# ============================================================================
# SECTION 7 : Création des DataLoaders
# ============================================================================

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

print(f"\n📚 DataLoaders créés:")
print(f"  Batches d'entraînement: {len(train_loader)}")
print(f"  Batches de validation: {len(val_loader)}")
print(f"  Batches de test: {len(test_loader)}")

# ============================================================================
# SECTION 8 : Initialisation du modèle DistilBERT
# ============================================================================

print("\n🤖 Chargement du modèle DistilBERT...")
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=NUM_LABELS
)
model = model.to(device)

# Optimiseur AdamW
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

# Learning rate scheduler
total_steps = len(train_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

print(f"✅ Modèle chargé sur {device}")
print(f"📈 Total steps: {total_steps}")

# ============================================================================
# SECTION 9 : Fonctions d'entraînement et d'évaluation
# ============================================================================

def train_epoch(model, data_loader, optimizer, scheduler, device):
    """Entraînement pour une époque"""
    model.train()
    losses = []
    correct_predictions = 0

    progress_bar = tqdm(data_loader, desc='Training')

    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # Forward pass
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        logits = outputs.logits

        # Calcul de l'accuracy
        _, preds = torch.max(logits, dim=1)
        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())

        # Backward pass
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        # Mise à jour de la progress bar
        progress_bar.set_postfix({'loss': loss.item()})

    accuracy = correct_predictions.double() / len(data_loader.dataset)
    avg_loss = np.mean(losses)

    return accuracy, avg_loss


def eval_model(model, data_loader, device):
    """Évaluation du modèle"""
    model.eval()
    losses = []
    correct_predictions = 0

    with torch.no_grad():
        for batch in tqdm(data_loader, desc='Evaluating'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            loss = outputs.loss
            logits = outputs.logits

            _, preds = torch.max(logits, dim=1)
            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())

    accuracy = correct_predictions.double() / len(data_loader.dataset)
    avg_loss = np.mean(losses)

    return accuracy, avg_loss

print("\n✅ Fonctions d'entraînement définies")

# ============================================================================
# SECTION 10 : Boucle d'entraînement
# ============================================================================

print("\n🚀 Début de l'entraînement...\n")

history = {
    'train_acc': [],
    'train_loss': [],
    'val_acc': [],
    'val_loss': []
}

best_accuracy = 0

for epoch in range(EPOCHS):
    print(f'{"="*60}')
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print(f'{"="*60}')

    # Entraînement
    train_acc, train_loss = train_epoch(
        model,
        train_loader,
        optimizer,
        scheduler,
        device
    )

    print(f'\n📊 Train - Loss: {train_loss:.4f} | Accuracy: {train_acc:.4f}')

    # Validation
    val_acc, val_loss = eval_model(model, val_loader, device)

    print(f'📊 Val   - Loss: {val_loss:.4f} | Accuracy: {val_acc:.4f}\n')

    # Sauvegarde de l'historique
    history['train_acc'].append(train_acc.cpu().numpy())
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc.cpu().numpy())
    history['val_loss'].append(val_loss)

    # Sauvegarde du meilleur modèle
    if val_acc > best_accuracy:
        torch.save(model.state_dict(), 'best_distilbert_model.bin')
        best_accuracy = val_acc
        print(f'💾 Meilleur modèle sauvegardé! Accuracy: {best_accuracy:.4f}')

print("\n✅ Entraînement terminé!")

# ============================================================================
# SECTION 11 : Visualisation des courbes d'entraînement
# ============================================================================

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Graphique Accuracy
ax1.plot(range(1, EPOCHS+1), history['train_acc'], 'b-o', label='Train', linewidth=2)
ax1.plot(range(1, EPOCHS+1), history['val_acc'], 'r-o', label='Validation', linewidth=2)
ax1.set_title('Accuracy par époque', fontsize=14, fontweight='bold')
ax1.set_xlabel('Époque', fontsize=12)
ax1.set_ylabel('Accuracy', fontsize=12)
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)
ax1.set_xticks(range(1, EPOCHS+1))

# Graphique Loss
ax2.plot(range(1, EPOCHS+1), history['train_loss'], 'b-o', label='Train', linewidth=2)
ax2.plot(range(1, EPOCHS+1), history['val_loss'], 'r-o', label='Validation', linewidth=2)
ax2.set_title('Loss par époque', fontsize=14, fontweight='bold')
ax2.set_xlabel('Époque', fontsize=12)
ax2.set_ylabel('Loss', fontsize=12)
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)
ax2.set_xticks(range(1, EPOCHS+1))

plt.tight_layout()
plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n📊 Graphiques sauvegardés dans 'training_history.png'")

# ============================================================================
# SECTION 12 : Évaluation sur le test set
# ============================================================================

print("\n" + "="*60)
print("ÉVALUATION SUR LE TEST SET")
print("="*60)

# Charger le meilleur modèle
model.load_state_dict(torch.load('best_distilbert_model.bin'))
print("\n✅ Meilleur modèle chargé")

def get_predictions(model, data_loader, device):
    """Obtenir les prédictions du modèle"""
    model.eval()
    predictions = []
    real_values = []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc='Prédiction'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            _, preds = torch.max(outputs.logits, dim=1)

            predictions.extend(preds.cpu().numpy())
            real_values.extend(labels.cpu().numpy())

    return predictions, real_values

# Obtenir les prédictions
y_pred, y_test = get_predictions(model, test_loader, device)

# Calcul des métriques
test_accuracy = accuracy_score(y_test, y_pred)

print(f"\n🎯 Accuracy sur le test set: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

print("\n📋 Rapport de classification détaillé:")
print("="*60)
print(classification_report(
    y_test,
    y_pred,
    target_names=['Politics', 'Sport', 'Tech'],
    digits=4
))

# ============================================================================
# SECTION 13 : Matrice de confusion
# ============================================================================

cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=['Politics', 'Sport', 'Tech'],
    yticklabels=['Politics', 'Sport', 'Tech'],
    cbar_kws={'label': 'Nombre de prédictions'}
)
plt.title('Matrice de Confusion - DistilBERT', fontsize=16, fontweight='bold', pad=20)
plt.ylabel('Vraie classe', fontsize=12)
plt.xlabel('Classe prédite', fontsize=12)
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n📊 Matrice de confusion sauvegardée dans 'confusion_matrix.png'")

# ============================================================================
# SECTION 14 : Analyse par classe
# ============================================================================

print("\n" + "="*60)
print("ANALYSE DÉTAILLÉE PAR CLASSE")
print("="*60)

for i, label_name in enumerate(['Politics', 'Sport', 'Tech']):
    mask = np.array(y_test) == i
    class_accuracy = accuracy_score(
        np.array(y_test)[mask],
        np.array(y_pred)[mask]
    )
    total_samples = mask.sum()
    correct_samples = (np.array(y_test)[mask] == np.array(y_pred)[mask]).sum()

    print(f"\n📌 {label_name}:")
    print(f"   Échantillons: {total_samples}")
    print(f"   Correct: {correct_samples}")
    print(f"   Accuracy: {class_accuracy:.4f} ({class_accuracy*100:.2f}%)")

# ============================================================================
# SECTION 15 : Fonction de prédiction pour nouveaux textes
# ============================================================================

def predict_news(text, model, tokenizer, device, max_length=512):
    """
    Prédire la catégorie d'un article de presse

    Args:
        text (str): Texte de l'article
        model: Modèle DistilBERT entraîné
        tokenizer: Tokenizer DistilBERT
        device: Device (CPU/GPU)
        max_length (int): Longueur maximale des tokens

    Returns:
        tuple: (classe prédite, probabilités)
    """
    model.eval()

    # Tokenization
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    # Prédiction
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = torch.nn.functional.softmax(logits, dim=1)
        _, prediction = torch.max(logits, dim=1)

    predicted_class = label_names[prediction.item()]
    probabilities = {
        label_names[i]: probs[0][i].item()
        for i in range(NUM_LABELS)
    }

    return predicted_class, probabilities

print("\n✅ Fonction de prédiction définie")

# ============================================================================
# SECTION 16 : Test avec des exemples
# ============================================================================

print("\n" + "="*60)
print("TEST AVEC DES EXEMPLES")
print("="*60)

test_examples = [
    "The football team won the championship after a great match against their rivals.",
    "New artificial intelligence breakthrough could revolutionize computer vision technology.",
    "The government announced new economic policies to tackle inflation and unemployment."
]

for i, text in enumerate(test_examples, 1):
    print(f"\n📄 Exemple {i}:")
    print(f"Texte: {text[:100]}...")

    predicted_class, probabilities = predict_news(text, model, tokenizer, device)

    print(f"\n🎯 Prédiction: {predicted_class}")
    print(f"\n📊 Probabilités:")
    for label, prob in sorted(probabilities.items(), key=lambda x: x[1], reverse=True):
        print(f"   {label}: {prob:.4f} ({prob*100:.2f}%)")

print("\n" + "="*60)
print("✅ NOTEBOOK TERMINÉ AVEC SUCCÈS!")
print("="*60)